<h1>Building a Neural network from scratch</h1>
<p>We are going to use just numpy for calculations with matrices and matplotlib.pyplot for showing results on charts</p>

In [2]:
import numpy as np
import matplotlib.pyplot as plt

<h4>ReLU:</h4>
<p>We use ReLU to set all the negative inputs to zero and leave positive inputs unchanged.<br>
It's simplicity makes it computationally efficient, especially in large neural networks.<br>
ReLU introduces non-linearity into the network, allowing it to learn complex patterns.<br>
For z > 0 the gradient is constant (dReLu(z)/dz =1), which help us avoid the vanishing gradient problem (which is common in sigmoid/tanh). <br>
Also it sets some activations to zero, which can help with computational efficiency and reduce the risk of overfitting.</p>

In [1]:
def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return Z > 0

<h4>Softmax Function:</h4>
<p>It converts raw scores z(logits) into probabilities and also ensures that the probabilities sum to 1 across all classes.<br>
Exponentiating the logits ensures all values are positive and also Larger logits get amplified more than smaller logits.<br>
Dividing by the sum of all exponentiated logits ensures the output values are probabilities is also some kind of normalization.The subtraction of max(Z) prevents numerical overflow(e.g for large exponentials).</p>
<p>In code below axis=0 ensures softmax is computed for each column(e.g for each class in a multi-class classification setting).<br>
And Also keepdims=True ensures that the result sum has the same dimensions as expZ, allowing for element-wise division without broadcasting issues.</p>
<h5>fromula for it is:</h5>

$$\text{softmax}(z_i) = \frac{\exp(z_i)}{\sum_{j=1}^{n} \exp(z_j)}$$


In [3]:
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return expZ / np.sum(expZ, axis=0, keepdims=True)

<h4>Cross entropy loss function</h4>
<p>This function computes the cross-entropy loss, which measures the difference between the predicted probabilities (y_pred) and the true labels (y_true). It is commonly used in classification tasks, especially for multi-class classification with softmax outputs.</p>
<h5>fromula for this loss function is:</h5>

$$J = -\frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} y_k^{(i)} \log(\hat{y}_k^{(i)})$$

If yk = 1(the correct class), the loss for that sample is equal to:

$$L = -\log(\hat{y}_k)$$
Encourages hat{yk} (the predicted probability for the correct class) to approach 1.<br>
The small value 1e-8 is added to hat{yk} to prevent taking the logarithm of 0, which would result in an undefined value

<h5>When Combined with Softmax Output:</h5>
<p>After simplifying, the gradient of the combined softmax and cross-entropy loss is:<br></p>

$$\frac{\partial z_k^{(i)}}{\partial J} = \hat{y}_k^{(i)} - y_k^{(i)}$$
<p>And when we Stack the derivatives for all m examples into a matrix: </p>

$$\frac{\partial Z}{\partial J} = \hat{Y} - Y$$


In [4]:
def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[1]
    loss = -np.sum(y_true * np.log(y_pred + 1e-8)) / m
    return loss

def cross_entropy_derivative(y_true, y_pred):
    return y_pred - y_true

<h4>Initializing the weights and biases</h4>
<p>Proper initialization of weights is critical for training the network effectively, as it impacts the convergence speed and overall performance.<br>
Poor initialization (e.g., very large or very small weights) can cause gradients to shrink (vanish) or grow (explode) exponentially as they propagate through layers.<br>
We set a seed for NumPy's random number generator to ensure reproducibility.<br>
Using the same seed produces the same random numbers every time the code runs, which is useful for debugging or comparing results.<br>
For initializing weights we use He initialization.<br>
For each weight we generate random values from a standard normal distribution(N(0, 1)) and scale it by * np.sqrt(2 / layer_dims[l-1])<br>
Using He initialization ensures the variance of the activations is maintained as the signal passes through layers, avoiding the problems of vanishing or exploding gradients<br></p>
<p>We initialize the biases to zeros by np.zeros()<br>
This doesn’t break symmetry, unlike initializing weights to zeros (which would cause all neurons in the layer to learn identical features).</p>



In [ ]:
def initialize_parameters(layer_dims):
    np.random.seed(0)
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        parameters[f"W{l}"] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(2 / layer_dims[l-1])
        parameters[f"b{l}"] = np.zeros((layer_dims[l], 1))
    return parameters

<h4>Dropout</h4>
<p>Dropout is a regularization technique where a random subset of neurons is "dropped" (set to 0) during training. This prevents the network from becoming too reliant on specific neurons and helps generalization.<br>
the process works like below:<br>
if dropout is enabled (keep_prob < 1.0):<br>
We create a random matrix D where each element is True with probability keep_prob and False otherwise.<br>
Then we drop some neurons in A by element-wise multiplying A and D and after that we scale the remaining activations by dividing by keep_prob.This will help us to maintain the expected value of the activations.After all, we store the dropout mask D in the cache.<br>
<h5>There are many benefits using this regularization technique:</h5>
<p>By randomly deactivating neurons, dropout prevents the network from relying too much on specific neurons.<br>
Also this technique forces the network to learn redundant representations since no single neuron can dominate.<br></p>
